In [ ]:
from ipycanvas import MultiCanvas,hold_canvas
from ipywidgets import Image
import math
import random
import IPython.display
import time
from IPython.core.display import HTML
with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# Kalah Rules

### Game structure:
<p>The playing field consists of 6 houses per side and additionally one large house per player at the edge, the so-called storage. At the beginning of the game, six seeds are placed in each house except the storage.</p>

![title](images/Board.png)

### Gameplay:
<p>Alternately, a player selects a house on his side of the board. The player then places the seeds in the following houses in a counter-clockwise direction. If visited, a seed is placed in the own storage, the opponent's storage is left out. The player's turn ends if all seeds are placed.

There are some special rules, that apply if the last seed is placed in the player's storage or an empty player's house:
1. If the last seed is placed in the own storage, it is the player's turn again. 
2. If the last seed is placed in an empty house on the player's own side of the board, this seed and all the seeds in the opposite house are placed in the player's storage. In that case, it is the opponent's turn.</p>

### End of the game:
<p>The game ends when all the houses of one player have been emptied. The opposing player then places all remaining seed in his storage.</p>

### Object of the game:
<p>The winner is the player who has more seeds in his storage at the end of the game.</p>

Source: http://www.kalaha.de/kalaha.htm (9.11.2021)

# Kalah game definition

### Basic definition:
<p>We define the game G as a six-tuple as follows so that a computer is able to play Kalah.</p>
<br>
<center><em>G = &ltStates, s0, Players, nextStates, finished, utility></em></center>

The components have the following meanings:

1. **States** is a set that contains all possible states of the game Kalah. A state of the game is represented by a list containing two lists, representing the houses of the two players. The first six values of each list represent the number of seeds in the player's houses represented by the letters **{A, B, C, D, E, F}**. The seventh and last value stands for the number of seeds in the corresponding player's storage. The start state of the game is for example:

<center><em>[[6,6,6,6,6,6,0], [6,6,6,6,6,6,0]]</em></center>

2. **s0 $\varepsilon$ States** is the start state.
3. **Players** is a list that contains the players. Kalah is a game for exactly two players, therefore this game's **Players** list only contains two elements.
4. **nextStates** is a function which calculates a set of states that are reachable by one move from Player p in the state s. To do so, the function receives a state **s $\varepsilon$ States** and a player **p $\varepsilon$ Players**. The formula is given as follows:

<center><em>nextStates: States x Players &rarr; 2<sup>States</sup></em></center>

5. **finished** is a function that takes a state **s $\varepsilon$ States** and checks if the game is finished, meaning that one of the players has emptied all their houses . The formula is: 
<br><br>
<center><em>finished: States &rarr; B</em></center>
<br>
The function finished is used to compute a set Terminalstates that contains all states from a finished game. This set is defined as follows: 
<br><br>
<center><em>TerminalStates:= {s $\varepsilon$ States | finished(s)}</em></center>
<br>
6. **utility** is a function that calculates the value of the game for a player. Therefore, the function takes a state **s $\varepsilon$ TerminalStates** and a player **p $\varepsilon$ Players**. The value that the function returns is an element from the set **{-1, 0, 1}**. The player p has lost the game when the function returns -1. If the function returns 1, the player has won the game and if the value is 0, the game ends in a draw. The formula for the function is:

<center><em>utility: TerminalStates x Players &rarr; {-1, 0. 1}</em></center>

source: https://github.com/karlstroetmann/Artificial-Intelligence/blob/master/Lecture-Notes/artificial-intelligence.pdf, S.87, Abruf am 06.02.2022

# Classes

The Kalah game is implemented with the use of several classes, containing base class <tt>Kalah_Game</tt>, a <tt>Board</tt> class for displaying the UI and a <tt>Player</tt> class as well as several classes inheriting from this <tt>Player</tt> class:

- <tt>Human</tt>
- <tt>Random_KI</tt>

The following illustration shows a class diagram of all classes involved:

<img src="images/Kalah_Class_Diagram.png" alt="Kalah_Class_Diagram" width="600"/>

This image was created with [Creatly](https://creately.com/).

## Global Variables

Additionally to the classes, there is one global variable with the purpose to help converting the state of the game board, which is represented by a nested list, to letters. The letter representation is mainly for the UI, but also distributes to print and logging messages being easier to humanly comprehend.

In [ ]:
order = [['A','B','C','D','E','F','O'],
         ['a','b','c','d','e','f','o']]

# Player Class

#### Attributes:
- *number*
- *name*

#### Methods:

- *\_\_init__(number, name)*
- *\_\_str__()*
- *_available_house(own_house)*
- *choose_house(current_state)*

The class **Player** is the superclass which represents a Kalah game player. In Kalah there are exactly two players which compete against each other. 

The attribute *number* of a player has either the value 0 or 1 depending on the order in which the players take their turns.
If the number of the player is 0, they start the game by having the first turn.
Additionally, *number* represents the index of the state list which contains the player's list of house values.

The attribute *name* is a string which represents the player's name in the UI or print and logging messages.

### Method: __init__
The method *\_\_init__(number, name)* initializes the <tt>Player</tt> object and checks if the given *number* is either 0 or 1 and if the given *name* is a string. If one of these criteria is not met, an error message is raised.

### Method: __str__
The method *\_\_str__()* defines the *name* of the <tt>Player</tt> object as their string representative for print messages.

In [ ]:
class Player:
    
    def __init__(self, number, name):
        if number in [0,1]:
            self.number = number
        else:
            raise ValueError("Number of player must be 0 or 1!")
        if isinstance(name, str):
            self.name = name
        else:
            raise ValueError("Name must be a string!")
            
    def __str__(self):
        return self.name

### Method: _available_house

The private method *_available_house(own_house)* receives a list which represents the player's house and their store. The method returns a list with the indizes of all house which contain at least one seed (list indizes with a value higher than null).

In [ ]:
def _available_house(self, own_house):
    available = []
    for i in range(len(own_house)):
        if own_house[i] != 0:
            available.append(i)
    return available

Player._available_house = _available_house
del _available_house

### Method: choose_house

The method *choose_house(current_state)* receives the current state of the Kalah board and returns the index of the chosen house from the player's house list. This method differentiates the <tt>Player</tt> subclasses and is therefore differently implemented in each of them. There is no basic implementation of this method in the <tt>Player</tt> class. This is why in the class <tt>Kalah_Game</tt> only subclasses of <tt>Player</tt> are accepted as players. Therefore, this class is intended as an abstract class.

In [ ]:
def choose_house(self, current_state):
    pass

Player.choose_house = choose_house
del choose_house

## Tests of Player Class

### 1. Player number must be 0 or 1

In [ ]:
try:
    Player(2, "Player1")
except ValueError as e:
    print(e)

### 2. Player name must be a string

In [ ]:
try:
    Player(0, 3)
except ValueError as e:
    print(e)

### 3. Successful Player creation

In [ ]:
Player(0, "Player1")

## Human Player Class

The class <tt>Human</tt> inherits from <tt>Player</tt>.
It is used for humans playing against each other or against one of the KIs. 

### Method: choose_house
The method *choose_house(current_state)* is implemented as follows:

At first the own house are extracted from the game state and the available house indizes are calculated using the method *_available_house*. Afterwards the human player is asked to choose one of the available house via a input field. With limiting the player to the available house, it is easier to detect a correct input. If the player enters an unvalid value, they are asked to input a value again until a valid value is given. The index of the corresponding house is returned.

In [ ]:
class Human(Player):
    
    def choose_house(self, current_state):
        
        own_t,store = current_state[self.number][:6],current_state[self.number][6]
        available = self._available_house(own_t)
        
        i_string = "Choose one of the available house:\n"
        for i in available:
            i_string += f"{order[self.number][i]}, "
        
        choice_str = ""
        letter_numbers = {order[1][i]:i for i in range(6)}
        print(letter_numbers)
        while choice_str not in [k for k in letter_numbers if letter_numbers[k] in available]:
            choice_str = input(i_string[:-2]+"\n").lower()
        
        choice = letter_numbers[choice_str]
        IPython.display.clear_output()
        return choice

### Test of Human Player Class

In [ ]:
human = Human(1,"Hans")

In [ ]:
#human.choose_house([[6,6,6,6,6,6,0], [6,0,6,6,4,6,0]])

## Random_KI Player Class

The class <tt>Random_KI</tt> inherits from <tt>Player</tt>. It is a simple KI player implementation which chooses house at random.

The class <tt>Random_KI</tt> has a modified *\_\_init__* function which has an additional argument *seed* for setting a seed for the **random** library functions. Whenever the same seed is set, the <tt>random</tt> function creates the same random numbers or chooses the same random objects from a list. Adding this argument eases the logging and debbuging of <tt>Random_KI</tt> players.

In [ ]:
import random as rn

class Random_KI(Player):
    
    def __init__(self, number, name, seed):
        rn.seed(seed)
        super().__init__(number, name)

### Method: choose_house
The method *choose_house(current_state)* is implemented as follows:

At first the own house are extracted from the game state and the available house indizes are calculated using the method *_available_house*. From the list of available house indizes, one is chosen at random using the function **choice** from the library **random**. Afterwards, this value is returned.

In [ ]:
def choose_house(self, current_state):
    
    
    
    own_t,safe = current_state[self.number][:6],current_state[self.number][6]
    available = self._available_house(own_t)

    choice = rn.choice(available)

    return choice

Random_KI.choose_house = choose_house
del choose_house

### Test of Random_KI Player Class

In [ ]:
ki = Random_KI(0,"Rando", 1)

In [ ]:
ki.choose_house([[6,6,6,6,6,6,0], [6,6,6,6,6,6,0]])

# Kalah_Game Class

#### Attributes:
- *state*
- *players*
- *current_player*

#### Methods:

- *\_\_init__(players)*
- *_other_player(player_num)*
- *_show_state(state)*
- *show_state()*
- *draw_board()*
- *_draw_seeds()*
- *_draw_numbers()*
- *_move(choice)*
- *_finished()*
- *utility(player_num)*
- *start()*

The class <tt>Kalah_Game</tt> is the core of the Kalah game implementation. It contains all information on the game, including the current game state.

The attribute *state* represents the state of the Kalah board which is defined by the number of seeds laying in each of the player's house and their stores. It is implemented by a nested list which contains a list for the house on each of the two players' sides. The last index of each of the lists is the store of that player. At the start of the game, where the stores are empty and there are six seeds in every house, the implemented representation of the state for example is: [[6,6,6,6,6,6,0], [6,6,6,6,6,6,0]].

The attribute *players* is a list of the two players that play the game. They must be instances of a subclass of the <tt>Player</tt> class. The order in which they take turns is determined by their <tt>Player</tt> *number*: The player with the number 0 goes first.


The attribute *current_player* has either the value 0 or 1. It takes track of which player's turn it currently is. If *current_player* has the value 0 for example, it is the turn of the player which stands at index 0 of the *players* list and therefore also has the <tt>Player</tt>'s class attribute *number* of value 0.

### Method: __init__
The method *\_\_init__(players)* initializes the game by setting the *state* to the initial state (seen above), initializing a board object and setting the *players* list using the received "players" argument. Before setting the *players* list, the received list is checked for the number of items it contains (must be exactly 2) and if the items in the list are both instances of a <tt>Player</tt> subclass (but not of the class <tt>Player</tt> itself). In error case, matching error messages are raised.

In [ ]:
class Kalah_Game():
    
    def __init__(self, players):
        self.state = [[6,6,6,6,6,6,0], [6,6,6,6,6,6,0]]
        
        if len(players) != 2:
            raise ValueError("There must be exactly two players!")
        if not ((isinstance(players[0], Player) and type(players[0]) != Player) 
            and (isinstance(players[1], Player) and type(players[1]) != Player)):
            raise ValueError("Both players must be of instances of a subclass of the class Player!")
        if {players[0].number, players[1].number} != {0,1}:
            raise ValueError("One of the players must be self.number 0 and the other one self.number 1!")
        
        self.players = players
        self.current_player = 0

### Method: _other_player
The auxiliary method *_other_player(player_num)* returns the opponent's player number to a given player number. This means that it returns 1 for the input 0, and 0 for the input 1.

In [ ]:
def _other_player(self, player_num):
    return (player_num + 1) % 2

Kalah_Game._other_player = _other_player
del _other_player

### Methods: _show_state and show_state
The private method *_show_state(state)* creates a formatted string which represents the received state and prints it to the console. It can be used as an alternative to the **ipycanvas** game UI.

The method *show_state()* calls the private method *_show_state(state)* with the current game state (attribute *state*).

In [ ]:
def _show_state(self, state):
    s = f''

    s += f'{self.players[0].name}:\t\t\t'
    for j in range(6,-1,-1):
        s += f'{order[0][j]}: {state[0][j]}  '
    s += f'\n'

    s += f'{self.players[1].name}:\t\t\t'
    for j in range(7):
        s += f'{order[1][j]}: {state[1][j]}  '
    s += f'\n'

    print(s)

Kalah_Game._show_state = _show_state
del _show_state

In [ ]:
def show_state(self):
    self._show_state(self.state)
    
Kalah_Game.show_state = show_state
del show_state

### Method: draw_board
The method *draw_board()* sets the required positional values for the methods *_draw_seeds()* and *_draw_numbers()*.

It also translates the current state of the game to a dictionary so that the positional values can be related to the corresponding seed amount in the respective houses.

Additionally this function creates a MultiCanvas for the display of the board. The first layer is the background and the second layer is the foreground.

Afterwards this function calls the functions *_draw_seeds()* and *_draw_numbers()*.

Finally the finished canvas gets displayed.

In [ ]:
def draw_board(self):
    TOP_ROW_POSITION = (175,130)
    HOUSE_OFFSETS = (90,145)
    HOUSE_MAP={
        'F':TOP_ROW_POSITION,
        'E':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0],TOP_ROW_POSITION[1]),
        'D':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*2,TOP_ROW_POSITION[1]),
        'C':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*3,TOP_ROW_POSITION[1]),
        'B':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*4,TOP_ROW_POSITION[1]),
        'A':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*5,TOP_ROW_POSITION[1]),
        'a':(TOP_ROW_POSITION[0],TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'b':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0],TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'c':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*2,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'd':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*3,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'e':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*4,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'f':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*5,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
    }
    
    canvas = MultiCanvas(2,width=800, height=400)
    canvas[0].draw_image(Image.from_file('images/Board.png'),0,0)
    
    with hold_canvas(canvas):
        self._draw_seeds(canvas[1],HOUSE_MAP)
        self._draw_numbers(canvas[1],HOUSE_MAP)
    display(canvas)

Kalah_Game.draw_board = draw_board
del draw_board

### Method: _draw_seeds
The method *_draw_seeds()* reads the game of the state and accordingly draws seeds on the respective houses. Each seed gets drawn in a random position inside the house with an additional random rotation.

To place the seeds accurately we must consider the width and height of the image of one seed. To do so we subtract the width and height from the final position chosen.

In [ ]:
def _draw_seeds(self,canvas,housemap):
    HOUSE_RAD = 23
    HOUSE_OFFSETS = (90,145)
    seedsprites = [Image.from_file(f'images/Samen{i}.png') for i in range(1,4)]
    seed_width = 15
    seed_height = 15
    
    gamestate = self._state_to_dict()
    for house in gamestate:
        for seed in range(gamestate[house]):
            seed_sprite = seedsprites[random.randrange(3)]
            
            if house == 'O':
                centerX, centerY = (housemap['F'][0]-HOUSE_OFFSETS[0],random.randrange(housemap['F'][1],housemap['F'][1]+HOUSE_OFFSETS[1]))
            elif house == 'o':
                centerX, centerY = (housemap['F'][0]+HOUSE_OFFSETS[0]*6,random.randrange(housemap['F'][1],housemap['F'][1]+HOUSE_OFFSETS[1]))
            else:
                centerX, centerY = housemap[house]

            r = HOUSE_RAD * math.sqrt(random.random())
            theta = random.random() * 2 * math.pi
            x = centerX + r * math.cos(theta)
            y = centerY + r * math.sin(theta)
            canvas.save()

            #Rotate by random amount for variation
            canvas.translate(x, y)
            canvas.rotate(random.uniform(0.,math.pi))
            canvas.translate(-x, -y)
            canvas.draw_image(seed_sprite, x-seed_width, y-seed_height)
            canvas.restore()

Kalah_Game._draw_seeds = _draw_seeds
del _draw_seeds

### Method: _draw_numbers
The method *_draw_numbers()* reads the game of the state and accordingly draws numbers that represent the amount of seeds on the respective houses. 



In [ ]:
def _draw_numbers(self,canvas,housemap):
    NUMBER_OFFSET_Y_BOT = 50
    NUMBER_OFFSET_Y_TOP = 60
    HOUSE_OFFSETS = (90,145)
    housenumbermap = {
        'F': (housemap['F'][0],housemap['F'][1]+NUMBER_OFFSET_Y_TOP),
        'E': (housemap['E'][0],housemap['E'][1]+NUMBER_OFFSET_Y_TOP),
        'D': (housemap['D'][0],housemap['D'][1]+NUMBER_OFFSET_Y_TOP),
        'C': (housemap['C'][0],housemap['C'][1]+NUMBER_OFFSET_Y_TOP),
        'B': (housemap['B'][0],housemap['B'][1]+NUMBER_OFFSET_Y_TOP),
        'A': (housemap['A'][0],housemap['A'][1]+NUMBER_OFFSET_Y_TOP),
        'a': (housemap['a'][0],housemap['a'][1]-NUMBER_OFFSET_Y_BOT),
        'b': (housemap['b'][0],housemap['b'][1]-NUMBER_OFFSET_Y_BOT),
        'c': (housemap['c'][0],housemap['c'][1]-NUMBER_OFFSET_Y_BOT),
        'd': (housemap['d'][0],housemap['d'][1]-NUMBER_OFFSET_Y_BOT),
        'e': (housemap['e'][0],housemap['e'][1]-NUMBER_OFFSET_Y_BOT),
        'f': (housemap['f'][0],housemap['f'][1]-NUMBER_OFFSET_Y_BOT),
        'O': (housemap['F'][0]-HOUSE_OFFSETS[0],housemap['F'][1]+HOUSE_OFFSETS[1]+HOUSE_OFFSETS[1]/5),
        'o': (housemap['F'][0]+HOUSE_OFFSETS[0]*6,housemap['F'][1]+HOUSE_OFFSETS[1]+HOUSE_OFFSETS[1]/5),
        
    }
    gamestate = self._state_to_dict()
    canvas.font = '26px serif'
    canvas.text_align    = 'center'
#     canvas.text_baseline = 'middle'
    for key in housenumbermap:
        canvas.fill_text(gamestate[key],housenumbermap[key][0],housenumbermap[key][1])
    
        
Kalah_Game._draw_numbers = _draw_numbers
del _draw_numbers

### Method: _move
The private method *_move(choice)* receives the number of the current player and the house index they have chosen as a result of the <tt>Player</tt> method *choose_house*. It calculates the actions of the player's turn and returns the resulting new game state.

##### The calculations are implemented based on the following game rules:

1. The current player choses one of his house. The seeds from the chosen house are placed counterclockwise in the house of both players and in the store of the current player. The store of the opponent is left out. Then it is the turn of the opponent.


2. If the last seed is placed in the store of the current player, they get another move.


3. If the last seed is placed in an empty house of the current player the seed from this house and all seeds from the opposite house are place in the store of the current player. Then it's the opponent's turn.

In [ ]:
def _move(self, choice):
    new_state = self.state.copy()

    seeds = self.state[self.current_player][choice]
    new_state[self.current_player][choice] = 0

    c_house_player_num = self.current_player
    c_house_num = choice

    # Go through houses counterclockwise
    while (seeds > 0):
        c_house_num += 1

        # Skip opponent's store
        if(c_house_num == 6 and c_house_player_num != self.current_player):
            continue

        # Switch to houses of the other player
        if(c_house_num > 6):
            c_house_num = 0
            c_house_player_num = self._other_player(c_house_player_num)

        # Add seed to the currently visited house
        new_state[c_house_player_num][c_house_num] += 1
        seeds -= 1

    # Check for special rules after last seed was placed in own store or own empty house:
    if(c_house_player_num == self.current_player):

        # Rule: Another turn if last seed is placed in own store
        if(c_house_num == 6):
            # Give current player another turn
            print(f'Player {self.players[self.current_player].name} gets an extra turn!')
            # Twist the order beforehand so the standard game loop grants the extra turn automatically
            self.current_player = self._other_player(self.current_player)

        # Rule: Last seed is placed in empty house of current player
        elif(new_state[self.current_player][c_house_num] == 1):                
            # Collect all seeds to be rewarded and empty both houses
            receivedSeeds = new_state[self.current_player][c_house_num]
            receivedSeeds += new_state[self._other_player(self.current_player)][5 - c_house_num]
            new_state[self._other_player(self.current_player)][5 - c_house_num] = 0
            new_state[self.current_player][c_house_num] = 0
            # Award all the seeds to the current player's kalah
            new_state[self.current_player][6] += receivedSeeds
            print(f'Player {self.players[self.current_player].name} gets a steal for {receivedSeeds} Seeds!')

    return new_state

Kalah_Game._move = _move
del _move

### Method: _finished
The private method *_finished()* checks if one of the players has no seeds in their house left and is therefore unable to take another turn. If this is the case, the method returns True, otherwise it returns False. 

In [ ]:
def _finished(self):
    sum0 = sum(self.state[0][:-1])
    sum1 = sum(self.state[1][:-1])

    if(not (sum0 == 0 or sum1 == 0)):
        return False
    self.show_state()
    print("Finished Game!")
    return True

Kalah_Game._finished = _finished
del _finished

### Method: _state_to_dict
The private method *_state_to_dict()* transforms a gamestate into a dictionary.

In [ ]:
def _state_to_dict(self):
        d = {}
        for i in [0,1]:
            for j in range(7):
                d[order[i][j]] = self.state[i][j]
        return d
Kalah_Game._state_to_dict = _state_to_dict
del _state_to_dict

### Method: utility
The  method *utility(player_num)* receives the number of the current player and uses the current state to calculate the utility of the state for the player. The method compares the seeds from both stores and returns an Integer dependent on the result. If the current player has more seeds than the opponent the method returns 1. If they own less seeds than the opponent the method returns -1 and if it is a draw the number 0 is returned.     

In [ ]:
def utility(self, player_num):
    playerStore = self.state[player_num][6]
    opponentStore = self.state[self._other_player(player_num)][6]
    if(playerStore > opponentStore):
        return 1
    elif(playerStore == opponentStore):
        return 0
    else:
        return -1
    
Kalah_Game.utility = utility
del utility

### Method: start
The method *start()* starts the Kalah game. Until the game is finished (the method *_finished()* returns True), both players take turns, starting with the player with the number 0. At the start of each turn, the current game state is shown. Next, the current player chooses one of the house with the <tt>Player</tt> method *choose_house(current_state)*. Afterwards, this choice is handed to the private method *_move(player_num, choice)* which calculates the new game state. The attribute *state* is updated with this new game state. Then it is the turn of the other player. If the game is finished, the method *utility()* calculates which of the players wins the game and the result is printed to the console.

In [ ]:
def start(self):
    while(not self._finished()):
        #TODO: Turn Counter
        print("Current state:")
        # self.show_state()
        self.draw_board()
        print(f"Next is {self.players[self.current_player].name}'s turn.")
        
        time.sleep(0.05)
        choice = self.players[self.current_player].choose_house(self.state)
        print(f"{self.players[self.current_player].name} chose {order[self.current_player][choice]}")

        self.state = self._move(choice)

        self.current_player = self._other_player(self.current_player)

    won0 = self.utility(0)
    if(won0 == 1):
        print(f"{self.players[0]} wins!")
    elif(won0 == 0):
        print(f"{self.players[1]} wins!")
    else:
        print(f"Draw!")
    
Kalah_Game.start = start
del start

## Tests of Kalah_Game Class

### 1. There must be exactly two players

In [ ]:
try:
    game = Kalah_Game([Player(0,"Test")])
except ValueError as e:
    print(e)

### 2. Players must be instances of a Player sublass

In [ ]:
try:
    game = Kalah_Game([Player(0,"Test"), Player(1,"Test")])
except ValueError as e:
    print(e)

### 3. Player numbers must be 0 and 1

In [ ]:
try:
    game = Kalah_Game([Human(1,"Human"), Random_KI(1,"KI",2)])
except ValueError as e:
    print(e)

### 4. Successful Game creation with two KIs

In [ ]:
game = Kalah_Game([Random_KI(0,"KI1",1), Human(1,"Karl")])

In [ ]:
game.show_state()

In [ ]:
game.start()

In [ ]:
gameHuman = Kalah_Game([Random_KI(0,"KI2",1), Human(1,"Human")])

In [ ]:
gameHuman.show_state()

In [ ]:
gameHuman.start()